In [1]:
"""
NEED TO BLOCK COMPONENTS AND PHANTOMS SINCE THE 1020 3/4 IS NOW THE
1020 5/6
NEED TO UPDATE GXS WITH NEW NUMBERS AND UPDATED PUMP DESCRIPTION

THERE ARE 4 DIFFERENCES WITH THE CASE CHANGE
1- NO PERFORMANCE CHANGE
2- APPEARNACE CHANGE BECAUSE OF THE ADDED RIBS WHICH CHANGED 2 OTHER
COMPONENTS
3- 4 LONGER CAPSCREWS
4- SLIGHT CHANGE TO THE GASKET WITH THE SHIFT IN THE BOLT HOLES
NO CHANGE TO EXISTING KITS- REVISIONS WERE MADE TO THESE BOMS

1020 CASING (BUY COMPONENT) CHANGE KP
125# 91840054 CHANGED TO 99717392       # Not Found
250# 91840056 CHANGED TO 99717485       # Not Found

1020 PHANTOM CHANGE
250# 98441969 CHANGED TO 92758523       # Not Found
125# 96698939 CHANGED TO 92758528       # in KPbom.xlsx 

HARDWARE PHANTOM CHANGED
KP
GRADE 5 96698968 CHANGE TO 92758121     # in KPbom.xlsx
GRADE 8 96768950 CHANGE TO 92758767     # in KPbom.xlsx
KPVS
GRADE 5 99413027 CHANGE TO 92758120     # Not Found
GRADE 8 NEW NUMBER         92758769     # ??

BOMS THAT HAVE BEEN CHANGED ARE IN THE FILE
"""

'\nNEED TO BLOCK COMPONENTS AND PHANTOMS SINCE THE 1020 3/4 IS NOW THE\n1020 5/6\nNEED TO UPDATE GXS WITH NEW NUMBERS AND UPDATED PUMP DESCRIPTION\n\nTHERE ARE 4 DIFFERENCES WITH THE CASE CHANGE\n1- NO PERFORMANCE CHANGE\n2- APPEARNACE CHANGE BECAUSE OF THE ADDED RIBS WHICH CHANGED 2 OTHER\nCOMPONENTS\n3- 4 LONGER CAPSCREWS\n4- SLIGHT CHANGE TO THE GASKET WITH THE SHIFT IN THE BOLT HOLES\nNO CHANGE TO EXISTING KITS- REVISIONS WERE MADE TO THESE BOMS\n\n1020 CASING (BUY COMPONENT) CHANGE KP\n125# 91840054 CHANGED TO 99717392       # Not Found\n250# 91840056 CHANGED TO 99717485       # Not Found\n\n1020 PHANTOM CHANGE\n250# 98441969 CHANGED TO 92758523       # Not Found\n125# 96698939 CHANGED TO 92758528       # in KPbom.xlsx \n\nHARDWARE PHANTOM CHANGED\nKP\nGRADE 5 96698968 CHANGE TO 92758121     # in KPbom.xlsx\nGRADE 8 96768950 CHANGE TO 92758767     # in KPbom.xlsx\nKPVS\nGRADE 5 99413027 CHANGE TO 92758120     # Not Found\nGRADE 8 NEW NUMBER         92758769     # ??\n\nBOMS THAT H

In [2]:
import os
import openpyxl
import pandas as pd

In [3]:
partnumbers_to_change = {
    'BOM': ['91840054', '91840056', '98441969', '96698939', '96698968', '96768950', '99413027'],
    'Model' : ['1020-3/4','1020-3_4','Desc-1020-3_4-KP_4','Desc-1020-3_4-KP_3']
}

replacements = {
    'BOM' : ['99717392','99717485','92758523','92758528','92758121','92758767','92758120'],
    'Model': ['1020-5/6','1020-5_6','Desc-1020-5_6-KP_6','Desc-1020-5_6-KP_5']
}

In [4]:
myDir = r'C:\Users\104092\OneDrive - Grundfos\Documents\20-29 Areas\22 grundfos-express-tools\22.06 KP-KPV-KPVS Updates\PSDs'

for root, dirs, files in os.walk(myDir):
    for filename in files:
        filePath = os.path.join(myDir, filename)
        
        wb = openpyxl.load_workbook(filePath)

        for sheet in wb.worksheets:
            sheetname = sheet.title
            # print(f'Current Sheet: {sheetname}')

            if sheetname == 'Case':

                # Only use PSDs with valid Export Setup for IQ
                for cell in sheet['A1':'A8']:
                    if cell[0].value == '[START]':
                        # Find # rows in header, in case they are different per each tab.
                        data_start_row = cell[0].row
                                
                        # Read in Excel sheet to a dataframe
                        data = pd.read_excel(filePath, sheet_name=sheetname, header=data_start_row-2)
                        data.fillna('', inplace=True)
                        # data = data.iloc[data_start_row::]
                        data = data.astype(str)

                        #rename $ with blank in every column name
                        data.columns = data.columns.str.replace('', 'Unused')


c:\Programs\Python\Python3102\lib\site-packages\openpyxl\reader\drawings.py:29: UserWarning: DrawingML support is incomplete and limited to charts and images only. Shapes and drawings will be lost.
  warn("DrawingML support is incomplete and limited to charts and images only. Shapes and drawings will be lost.")
c:\Programs\Python\Python3102\lib\site-packages\openpyxl\reader\drawings.py:29: UserWarning: DrawingML support is incomplete and limited to charts and images only. Shapes and drawings will be lost.
  warn("DrawingML support is incomplete and limited to charts and images only. Shapes and drawings will be lost.")


original: 96698939 exists in PSD: KPbom.xlsx, sheet: Case
original: 96698968 exists in PSD: KPbom.xlsx, sheet: Hardware
original: 96768950 exists in PSD: KPbom.xlsx, sheet: Hardware


In [ ]:

                    '''Dont need to iterate through. Just replace with entire dictionary'''
                    for _, item_list in partnumbers_to_change.items(): 
                        # Check if old partnumbers exist in DataFrame
                        for item in item_list:
                            if item in data.values:
                                print(f'original: {item} exists in PSD: {filename}, sheet: {sheetname}')
                            
                    data.replace(partnumbers_to_change, replacements, inplace=True, regex=True)

                    ### Writing to excel file
                    # Write resulting dataframes to new sheet in PSD with changes.

                    with pd.ExcelWriter(filePath, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:  
                        data.to_excel(writer, sheet_name=sheetname, index=False, header=False, startrow=data_start_row-1)
                        # removals.to_excel(writer, sheet_name=tabName, index=False, header=False, startrow=append_location+1, startcol=psd_startcol)
